<a href="https://colab.research.google.com/github/hatkiet/Project_4/blob/Mia-branch/3_1b_HeartAttack_ML_RandomForests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
# Import required libraries and dependencies
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report

In [102]:
# Load the data into a Pandas DataFrame
df = pd.read_csv("heart_2022_cleaned.csv")
# Display sample data
df

,Sex,Age,Race,BMI,GenHealth,PhysicalHealth,MentalHealth,PhysicalActivity,ChestScan,SleepHours,...,ECigaretteUsage,Drinking,HeartAttack,Angina,Stroke,Asthma,SkinCancer,Depressed,KidneyDisease,Diabetes
0,0,12,0,26.57,4,0.0,0.0,0,0,6.0,...,0,0,0,0,0,0,0,0,0,0
1,0,7,0,25.61,3,2.0,3.0,1,0,5.0,...,0,0,0,0,0,0,0,0,0,0
2,0,4,0,21.77,1,2.0,0.0,1,1,9.0,...,0,1,0,0,0,0,1,0,0,0
3,1,12,0,26.08,0,1.0,0.0,0,0,7.0,...,0,0,1,0,1,0,1,0,0,1
4,0,12,1,22.96,3,0.0,0.0,1,0,7.0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318929,1,9,3,29.86,2,0.0,15.0,1,0,7.0,...,0,1,0,0,1,0,1,0,0,1
318930,1,2,0,31.19,2,0.0,0.0,1,0,8.0,...,0,1,0,0,0,0,1,0,0,0
318931,0,6,1,28.66,4,2.0,2.0,1,0,7.0,...,0,0,0,0,0,0,1,0,0,0
318932,1,10,1,32.55,3,0.0,0.0,0,1,5.0,...,0,0,1,0,0,1,1,0,0,0


In [103]:
# Copy dataset for Random Forests - Supervised Learning
df5 = df.copy()

In [104]:
# Define features set X
X = df5.drop("HeartAttack", axis=1)
y = df5["HeartAttack"].values.reshape(-1,1)
y[:5]

array([[0],
       [0],
       [0],
       [1],
       [0]])

In [105]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [106]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [107]:
# Encoding categorical features
X_train_encoded = pd.get_dummies(X_train, columns=["GenHealth"])

# Identify non-numerical columns
non_numeric_cols = X_train_encoded.select_dtypes(exclude=['number']).columns

# Convert non-numerical columns to numerical (example using LabelEncoder)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for col in non_numeric_cols:
    X_train_encoded[col] = label_encoder.fit_transform(X_train_encoded[col])

# Fitting Standard Scaler after encoding
X_scaler = scaler.fit(X_train_encoded)

In [108]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train_encoded) # Use the encoded training data

# Encoding and scaling the test data
X_test_encoded = pd.get_dummies(X_test, columns=["GenHealth"]) # One-hot encode the test data
non_numeric_cols_test = X_test_encoded.select_dtypes(exclude=['number']).columns
for col in non_numeric_cols_test:
    X_test_encoded[col] = label_encoder.fit_transform(X_test_encoded[col]) # Use the same label encoder as for training data
X_test_scaled = X_scaler.transform(X_test_encoded) # Transform the encoded test data

In [109]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [110]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [111]:
rf_model.score(X_train_scaled, y_train)

0.9997742474916388

In [112]:
rf_model.score(X_test_encoded, y_test)

0.9470614794190684

In [113]:
rf_model.score(X_test_scaled, y_test)

0.9490305265006146

In [114]:
prep = rf_model.predict(X_test_scaled)
prep

array([0, 1, 0, ..., 0, 0, 0])

In [115]:
accuracy_score(y_test, prep)

0.9490305265006146

In [116]:
print(classification_report(y_test, prep))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97     75513
           1       0.54      0.23      0.32      4221

    accuracy                           0.95     79734
   macro avg       0.75      0.61      0.65     79734
weighted avg       0.94      0.95      0.94     79734



In [117]:
df5["HeartAttack"].value_counts()

HeartAttack
0    301988
1     16946
Name: count, dtype: int64

In [118]:
X = df5.drop("HeartAttack",axis=1)
y = df5.HeartAttack
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=78)
X_res, y_res = rus.fit_resample(X, y)

In [119]:
print(y_res.value_counts())

HeartAttack
0    16946
1    16946
Name: count, dtype: int64


In [120]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
# Fitting the model
rf_model = rf_model.fit(X_res, y_res)
print(f'Model score: {rf_model.score(X_res, y_res)}')
prep = rf_model.predict(X_res)

print(f'Accuracy Score: {accuracy_score(y_res, prep)}')
print('')
print(classification_report(y_res, prep))
print('')
print("Confusion Matrix:\n", confusion_matrix(y_res, prep))
print('')

Model score: 0.9997639560958338
Accuracy Score: 0.9997639560958338

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16946
           1       1.00      1.00      1.00     16946

    accuracy                           1.00     33892
   macro avg       1.00      1.00      1.00     33892
weighted avg       1.00      1.00      1.00     33892


Confusion Matrix:
 [[16945     1]
 [    7 16939]]



In [121]:
pred_df = X_res.copy()
pred_df["Actual"] = y_res
pred_df["Predicted"] = prep
pred_df

,Sex,Age,Race,BMI,GenHealth,PhysicalHealth,MentalHealth,PhysicalActivity,ChestScan,SleepHours,...,Drinking,Angina,Stroke,Asthma,SkinCancer,Depressed,KidneyDisease,Diabetes,Actual,Predicted
0,1,3,4,25.40,3,2.0,20.0,1,1,6.0,...,1,0,1,0,1,1,0,0,0,0
1,0,9,0,36.05,2,0.0,0.0,1,1,6.0,...,0,0,0,0,1,0,0,0,0,0
2,0,7,0,27.46,3,3.0,3.0,1,0,5.0,...,1,0,0,0,1,0,0,0,0,0
3,0,0,4,20.91,2,0.0,10.0,1,0,6.0,...,0,0,0,0,1,0,0,0,0,0
4,0,9,0,29.05,2,2.0,0.0,0,1,6.0,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33887,1,3,1,33.38,1,0.0,0.0,1,0,8.0,...,0,0,1,1,1,0,0,0,1,1
33888,1,12,4,24.81,3,0.0,10.0,1,0,7.0,...,1,1,0,0,1,0,0,0,1,1
33889,1,9,1,33.38,1,7.0,30.0,0,1,4.0,...,1,1,0,0,1,0,0,0,1,1
33890,1,7,1,36.54,2,0.0,0.0,0,1,6.0,...,1,0,0,0,1,0,0,0,1,1


In [122]:
df2.columns

NameError: name 'df2' is not defined